In [1]:
import sqlite3
import numpy as np
import mysql.connector
import pandas as pd
import getpass
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
plotly.tools.set_credentials_file(username='mainakchain', api_key='B9m4DVrYYmsoW3jDSiok')
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
from scipy import stats
from sklearn.utils import shuffle
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
import datetime
import holidays
from sklearn.model_selection import cross_val_score
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVR
from sklearn.svm import SVR

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [31]:
toll_time = pd.read_csv('toll_time.csv')

In [32]:
toll_time = toll_time[toll_time['origin'] == 'Jamshedpur'][toll_time['destination'] == 'HOSUR']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [33]:
toll_time.drop(['Unnamed: 0'], axis=1, inplace=True)

In [34]:
new_toll_time = toll_time.copy()

In [35]:
today_date = datetime.datetime.now()
today_date.strftime("%Y-%m-%d %H:%M:%S")
today = today_date.strftime("%Y-%m-%d %H:%M:%S")
new_toll_time['today'] = datetime.datetime.strptime(today, '%Y-%m-%d %H:%M:%S')

In [36]:
new_toll_time['start_date'] = new_toll_time['start_date'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['days_from_now'] = new_toll_time['today'] - new_toll_time['start_date'] 
new_toll_time['days_from_now'] = new_toll_time['days_from_now'].apply(lambda x: x.days)

In [39]:
new_toll_time = toll_time.copy()

In [40]:
today_date = datetime.datetime.now()
today_date.strftime("%Y-%m-%d %H:%M:%S")
today = today_date.strftime("%Y-%m-%d %H:%M:%S")
new_toll_time['today'] = datetime.datetime.strptime(today, '%Y-%m-%d %H:%M:%S')

new_toll_time['start_date'] = new_toll_time['start_date'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['days_from_now'] = new_toll_time['today'] - new_toll_time['start_date'] 
new_toll_time['days_from_now'] = new_toll_time['days_from_now'].apply(lambda x: x.days)

new_toll_time['unloading_in_time'] = new_toll_time['unloading_in_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['avg_travel_time'] = new_toll_time['start_date'] + (new_toll_time['unloading_in_time'] - new_toll_time['start_date'])/2

new_toll_time['month_of_travel'] = new_toll_time['avg_travel_time'].apply(lambda x: x.month)

new_toll_time['ist_timestamp'] = new_toll_time['ist_timestamp'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
new_toll_time['time_of_travel'] = new_toll_time.ist_timestamp.apply(lambda x: x.hour)

new_toll_time['day_of_travel'] = new_toll_time['avg_travel_time'].apply(lambda x: x.weekday())

new_toll_time['loading_out_time'] = new_toll_time['loading_out_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['loading_out_time'] = new_toll_time['loading_out_time'].apply(lambda x: x.hour)

In [41]:
new_toll_time = new_toll_time[['toll_both','vehicle_no','loading_out_time','distance_travelled','time_of_travel','day_of_travel','month_of_travel','days_from_now','time_taken']]

In [119]:
train_df = new_toll_time.copy()
test_df= pd.DataFrame(columns=['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now'])
choice = True
while(choice):
    today_date = datetime.datetime.now()
    vehicle = raw_input('Enter the vehicle no.: ')
    start = raw_input('Enter the start date in %Y-%m-%d %H:%M:%S format: ')
    start_delta = datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    
    loading_out = raw_input('Enter the loading out time in %Y-%m-%d %H:%M:%S format: ')
    loading_out_delta = datetime.datetime.strptime(loading_out, '%Y-%m-%d %H:%M:%S')
    
    a = []
    loop = input('Enter the number of tolls vehicle crossed sequentitially: ')
    count = 0
    print '      '
    while (count != loop):
        b = []
        toll = raw_input('Toll both: ')
        distance = input('Distance travelled from 1st toll in km: ')
        ist = raw_input('IST Timestamp in %Y-%m-%d %H:%M:%S format: ')
        ist_delta = datetime.datetime.strptime(ist, '%Y-%m-%d %H:%M:%S')
        b.extend([toll, vehicle, loading_out_delta.hour, distance, ist_delta.hour, ist_delta.weekday(), ist_delta.month,
                      (today_date - loading_out_delta).days, (ist_delta - loading_out_delta).total_seconds()/3600])
        a.append(b)
        count+=1
    
    add_df = pd.DataFrame(a, columns= ['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now', 'time_taken'])
    train_df = train_df.append(add_df)
    
    test_toll = raw_input('Enter the numbers of tolls the vehicle shall take in the journey sequentially: ')
    test_toll_list = test_toll.split(',')
    test_toll_list = [str(a) for a in test_toll_list]

    a = []

    for toll_val in test_toll_list:
        b = []
        distance = toll_time['distance_travelled'][toll_time.toll_both == toll_val].median()
        ist_delta = loading_out_delta + timedelta(hours=toll_time['time_taken'][toll_time.toll_both == toll_val].median())
        b.extend([toll_val, vehicle, loading_out_delta.hour, distance, ist_delta.hour, ist_delta.weekday(), ist_delta.month,
                      (today_date - loading_out_delta).days])

        a.append(b)
    add_test_df = pd.DataFrame(a, columns= ['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now'])
    
    test_df = test_df.append(add_test_df)
    
    c = raw_input('Want to enter more vehicles?(y/n)')
    
    if (c == 'y' or c == 'Y'):
        continue
    elif (c == 'n' or c == 'N'):
        choice = False
        print 'bbye'
    else:
        print 'Wrong Choice! Exiting'
        choice = False

Enter the vehicle no.: OR04M532M
Enter the start date in %Y-%m-%d %H:%M:%S format: 2017-11-02 00:00:00
Enter the loading out time in %Y-%m-%d %H:%M:%S format: 2017-11-03 12:02:43
Enter the number of tolls vehicle crossed sequentitially: 3
      
Toll both: Sergrarh
Distance travelled from 1st toll in km: 0
IST Timestamp in %Y-%m-%d %H:%M:%S format: 2017-11-04 08:22:51
Toll both: Panikholi
Distance travelled from 1st toll in km: 95.1
IST Timestamp in %Y-%m-%d %H:%M:%S format: 2017-11-04 10:42:12
Toll both: Manguli
Distance travelled from 1st toll in km: 146.8
IST Timestamp in %Y-%m-%d %H:%M:%S format: 2017-11-04 12:10:12
Enter the numbers of tolls the vehicle shall take in the journey sequentially: Gudipada (Old Gangapada),Gurapalli,Bellupada,Madapam,Chilakapalem,Nathavalasa/ Vizianagaram,Laxmipuram,Vempadu,Krishnavaram,Kalaparru,Pottipadu,Kaza,Bolapalli,Sunambatti (Musunur),Venkatachallam &#160;(Old Nellore),Budanam,Gaddurur,Mulbagal,Hoskote,Attibele (BETL)
Want to enter more vehicles?

In [6]:
train_df = pd.read_csv('final_2nd_train_df.csv')
test_df = pd.read_csv('final_2nd_test_df.csv')


In [3]:
train_df.tail(10)

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now
28,11,2,21,918.860,11,2,10,52
29,12,2,21,964.230,13,2,10,52
30,13,2,21,1046.730,17,2,10,52
31,14,2,21,1171.885,3,3,10,52
32,15,2,21,1239.060,6,3,10,52
33,16,2,21,1269.870,7,3,10,52
34,17,2,21,1487.190,23,3,10,52
35,18,2,21,1517.610,0,4,10,52
36,19,2,21,1577.280,18,3,10,52
37,20,2,21,1615.100,18,2,10,52


In [121]:
test_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now
0,Gudipada (Old Gangapada),OR04M532M,12,203.320,10,6,11,40
1,Gurapalli,OR04M532M,12,291.250,13,6,11,40
2,Bellupada,OR04M532M,12,375.245,20,6,11,40
3,Madapam,OR04M532M,12,490.530,0,0,11,40
4,Chilakapalem,OR04M532M,12,517.415,3,0,11,40
5,Nathavalasa/ Vizianagaram,OR04M532M,12,556.620,5,0,11,40
6,Laxmipuram,OR04M532M,12,556.620,5,0,11,40
7,Vempadu,OR04M532M,12,684.270,11,0,11,40
8,Krishnavaram,OR04M532M,12,753.930,16,0,11,40
9,Kalaparru,OR04M532M,12,897.510,2,1,11,40


In [122]:
train_df.shape, test_df.shape

((998, 9), (38, 8))

In [132]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [159]:
def change_toll_booth_to_dummy(df):
    num = 1
    dict_toll_both = {}
    for toll in df['toll_both'].unique().tolist():
        df['toll_both'] = df['toll_both'].replace({toll: num})
        dict_toll_both[num] = toll
        num += 1
    return dict_toll_both

In [160]:
dict_toll_both_train_df = change_toll_booth_to_dummy(train_df)
dict_toll_both_test_df = change_toll_booth_to_dummy(test_df)

In [161]:
def change_vehicle_no_to_dummy(df):
    num = 1
    dict_vehicle_no = {}
    for truck in df['vehicle_no'].unique().tolist():
        df['vehicle_no'] = df['vehicle_no'].replace({truck: num})
        dict_vehicle_no[num] = truck
        num += 1
    return dict_vehicle_no

In [162]:
dict_vehicle_no_train_df = change_vehicle_no_to_dummy(train_df)
dict_vehicle_no_test_df = change_vehicle_no_to_dummy(test_df)

In [163]:
# MODEL BUILDING AND APPLICATION

In [5]:
train_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now
0,1,1,12,203.320,10,6,11,40
1,2,1,12,291.250,13,6,11,40
2,3,1,12,375.245,20,6,11,40
3,4,1,12,490.530,0,0,11,40
4,5,1,12,517.415,3,0,11,40
5,6,1,12,556.620,5,0,11,40
6,7,1,12,556.620,5,0,11,40
7,8,1,12,684.270,11,0,11,40
8,9,1,12,753.930,16,0,11,40
9,10,1,12,897.510,2,1,11,40


In [ ]:
train_X['avg_speed'] = train_X['distance_travelled']/train_X['tim']

In [7]:
train_X = train_df.drop(['time_taken'], axis=1)
train_y = train_df.time_taken

In [8]:
train_X.shape

(998, 8)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(train_X,train_y, test_size=0.3, random_state=42)

In [149]:
reg_mlp = MLPRegressor(hidden_layer_sizes=(100,50), activation='logistic', alpha=8, solver='lbfgs', random_state=42)

In [152]:
scale= MinMaxScaler()
X_train_scaled = scale.fit_transform(X_train)
X_test_scaled = scale.fit_transform(X_test)

In [154]:
reg_mlp.fit(X_train_scaled, y_train)
reg_mlp.score(X_test_scaled, y_test) * 100

90.5241146301952

In [155]:
scores = cross_val_score(reg_mlp, X_train_scaled, y_train, scoring='r2', n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.83 (+/- 0.03)


In [182]:
scale = MinMaxScaler()
X_predict_scaled = scale.fit_transform(test_df)

In [183]:
predictions = reg_mlp.predict(X_predict_scaled)

In [184]:
show_df = test_df.copy()
show_df['predicted'] = predictions

In [190]:
show_df['predict_diff'] = show_df['predicted'].diff()

In [192]:
def med_toll_time_taken(toll):
    return train_df['time_taken'][train_df['toll_both'] == toll ].median()

In [199]:
def mean_toll_time_taken(toll):
    return train_df['time_taken'][train_df['toll_both'] == toll ].mean()

In [200]:
show_df['med_time'] = show_df['toll_both']
show_df['med_time'] = show_df.med_time.apply(lambda x:med_toll_time_taken(x))
show_df['mean_time'] = show_df['toll_both']
show_df['mean_time'] = show_df.mean_time.apply(lambda x:mean_toll_time_taken(x))

In [203]:
show_df['avg_pred_med'] = (show_df['predicted'] + show_df['med_time']) / 2

In [214]:
x = train_df[train_df['vehicle_no'] == 10]
x = x.sort_values('distance_travelled')
x['avg_speed'] = x.distance_travelled/ x.time_taken
x

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken,avg_speed
770,27,10,19,0.00,16,5,11,21,20.705556,0.000000
680,25,10,19,93.64,18,5,11,21,23.476111,3.988736
550,20,10,19,147.37,20,5,11,21,25.079167,5.876192
210,9,10,19,203.44,22,5,11,21,26.732222,7.610291
255,10,10,19,292.16,1,5,11,21,29.935278,9.759722
9,3,10,19,376.19,3,5,11,21,31.972222,11.766151
503,18,10,19,492.18,7,5,11,21,35.745000,13.769198
130,6,10,19,519.32,7,5,11,21,36.345000,14.288623
459,17,10,19,559.10,9,5,11,21,37.867500,14.764640
632,22,10,19,559.10,9,5,11,21,37.867500,14.764640


In [213]:
show_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,predicted,predict_diff,med_time,mean_time,avg_pred_med
0,1,1,12,203.320,10,6,11,40,60.305124,NaN,84.090278,84.090278,72.197701
1,2,1,12,291.250,13,6,11,40,62.938448,2.633323,93.539722,112.452738,78.239085
2,3,1,12,375.245,20,6,11,40,64.313343,1.374895,56.755556,61.037500,60.534449
3,4,1,12,490.530,0,0,11,40,58.601461,-5.711881,92.291944,99.164912,75.446703
4,5,1,12,517.415,3,0,11,40,59.647295,1.045834,106.137500,113.508902,82.892398
5,6,1,12,556.620,5,0,11,40,61.671978,2.024683,63.638889,66.576692,62.655434
6,7,1,12,556.620,5,0,11,40,61.477310,-0.194668,48.963056,48.963056,55.220183
7,8,1,12,684.270,11,0,11,40,68.672551,7.195241,122.277500,130.865833,95.475025
8,9,1,12,753.930,16,0,11,40,72.610093,3.937542,45.996250,50.308219,59.303171
9,10,1,12,897.510,2,1,11,40,65.359971,-7.250122,49.686667,52.731903,57.523319
